In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [4]:
from proj1_helpers import load_csv_data
DATA_TRAIN_PATH = './data/train.csv'
y, x, ids = load_csv_data(DATA_TRAIN_PATH)
print(y)
## Note: This is the raw dataset, you can also work with your modified features if you prefer

[ 1. -1. -1. ...,  1. -1. -1.]


In [71]:
def hinge_loss(z):
    return max(0,z)

def calculate_cost(y, x, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer."""
    # Here x is the full dataset matrix, and y are the corresponding +1 or -1 labels
    z = np.sum(hinge_loss(1-y.dot(x).dot(w))+lambda_/2*np.linalg.norm(w)**2)
    return z

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [72]:
def calculate_gradient(y, x, w, lambda_, n):
    """compute the stochastic gradient of loss plus regularizer."""
    # Here x is one datapoint, and y is the corresponding +1 or -1 label
    x_col = x[n,:].reshape(w.shape)
    a = 1-y[n]*x_col.T.dot(w)
    if hinge_loss(a[0]) <= 0.00001:
        return np.zeros(w.shape)
    return -y[n]*(x_col)+lambda_*w  #derivative of cost
    # Be careful about the constant N(size) term! The complete objective for SVM is a sum, not an average as in earlier SGD examples!

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [76]:
import random

def sgd_for_svm_demo(y, x):
    # ***************************************************
    # INSERT YOUR CODE HERE
    # classify the data by SGD for SVM: TODO
    # ***************************************************
    max_iter = 5000
    gamma = 0.001
    lambda_ = 1.0 / y.shape[0]  # or set to a different value, try cross-validation!
    N = len(y)
    w = np.zeros((x.shape[1], 1))
    
    for iter in range(max_iter):
        #n = sample one data point uniformly at random data from x
        n = random.randint(0, N-1)
        loss = calculate_cost(y, x, w, lambda_) 
        grad = calculate_gradient(y, x, w, lambda_, n)
        w = w - gamma*grad        
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
    
    print("Objective = {l}".format(l=calculate_cost(y, x, w, lambda_)))

sgd_for_svm_demo(y, x)

Current iteration=0, the loss=1.0
Current iteration=100, the loss=1.9823473647793265e-05
Current iteration=200, the loss=5.6979754598695945e-05
Current iteration=300, the loss=90254186.42429535
Current iteration=400, the loss=4.744136584339667e-05
Current iteration=500, the loss=4.058478073656871e-05
Current iteration=600, the loss=0.00010469881504377111
Current iteration=700, the loss=5.7827199581732184e-05
Current iteration=800, the loss=215133325.54571608
Current iteration=900, the loss=0.00016296322569204978
Current iteration=1000, the loss=0.00013761313410617365
Current iteration=1100, the loss=0.0001284879525856357
Current iteration=1200, the loss=0.00014298961374285192
Current iteration=1300, the loss=0.00012434091851067875
Current iteration=1400, the loss=0.00019521438706902205
Current iteration=1500, the loss=0.00018147173540699346
Current iteration=1600, the loss=0.00019664035271023305
Current iteration=1700, the loss=182746672.36601412
Current iteration=1800, the loss=0.0002

## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [77]:
def calculate_coordinate_update(y, x, lambda_, alpha, w, n):
    # Here x is one datapoint, and y is the corresponding +1 or -1 label
    # n is the coordinate
    #slide 6 (find alpha that maximizes g)
    #Gradient: np.ones(y.shape) - 1/lambda_*x.dot(Y).dot(alpha) #derivative of XAX.T = 2AX (Francesco)
    #setting gradient to 0:
    return alpha[n]-1/lambda_ * y[n]*(x[n].T).dot(alpha)

In [ ]:
def coordinate_descent_for_svm_demo(y, x):
    # ***************************************************
    # classify the data by SGD for SVM: TODO
    # ***************************************************
    max_iter = 10000
    gamma = 0.001
    lambda_ = 1.0 / y.shape[0]
    N = len(y)
    w = np.zeros((x.shape[1], 1))
    
    for iter in range(max_iter):
        # n = uniformly random data point from x
        n = random.randint(0, N-1)
        loss = calculate_cost(y, x, w, lambda_) 
        #we're maximizing! gradient ascent,  slide 9
        coords = calculate_coordinate_update(y, x, lambda_, alpha, w)
        alpha[n] = coords[n]
        Y = np.diagflat(y)
        w = update1/lambda_ * x.dot(Y).dot(alpha) #slide 6, w(alpha)
        
        if iter % 1000 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
    
    print("Primal objective = {l}".format(l=calculate_cost(y, x, w, lambda_)))

coordinate_descent_for_svm_demo(y, x)